In [14]:
!unzip tennis_court_det_dataset.zip

unzip:  cannot find or open tennis_court_det_dataset.zip, tennis_court_det_dataset.zip.zip or tennis_court_det_dataset.zip.ZIP.


# Start Code

In [169]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np
import os

In [170]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create torch dataset

In [184]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        # img_path = os.path.join(self.img_dir, f"{item['id']}.png")
        # if not os.path.exists(img_path):
        #     raise FileNotFoundError(f"Tệp ảnh '{img_path}' không được tìm thấy.")
        
        # img = cv2.imread(img_path)
        
        # # Kiểm tra xem ảnh có được tải thành công hay không
        # if img is None:
        #     raise ValueError(f"Không thể tải ảnh từ '{img_path}'.")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps
    

In [173]:
train_dataset = KeypointsDataset("data/images", "/Users/phungtatdat/pythonCode/tennis-analysis/data/data_train.json")
val_dataset = KeypointsDataset("data/images", "/Users/phungtatdat/pythonCode/tennis-analysis/data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_laoder = DataLoader(val_dataset, batch_size=8, shuffle=True)

#Create model

In [174]:
model = models.resnet50() #pretrained=True
model.fc = torch.nn.Linear(model.fc.in_features, 14*2) #Relace the last layer

In [175]:
model = model.to(device)

# Train the model

In [176]:
criterion = torch.nn.MSELoss()  
optimizer= torch.optim.Adam(model.parameters(), lr=1e-4)
   

In [185]:
epochs= 20
for epoch in range(epochs):
    for i, (imgs, kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)
        
        optimizer.zero_grad()
        outputs=model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, Step {i+1}/{len(train_loader)} iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 12233.3916015625
Epoch 0, iter 10, loss: 11160.0556640625
Epoch 0, iter 20, loss: 10674.8134765625
Epoch 0, iter 30, loss: 9777.0908203125
Epoch 0, iter 40, loss: 9107.7353515625
Epoch 0, iter 50, loss: 8420.142578125
Epoch 0, iter 60, loss: 7659.8291015625
Epoch 0, iter 70, loss: 7283.375
Epoch 0, iter 80, loss: 6381.7421875
Epoch 0, iter 90, loss: 5704.62744140625
Epoch 0, iter 100, loss: 5192.94775390625
Epoch 0, iter 110, loss: 4918.24658203125
Epoch 0, iter 120, loss: 4445.83544921875
Epoch 0, iter 130, loss: 4345.56982421875
Epoch 0, iter 140, loss: 3441.156982421875
Epoch 0, iter 150, loss: 3117.034423828125
Epoch 0, iter 160, loss: 2908.01953125
Epoch 0, iter 170, loss: 3171.428466796875
Epoch 0, iter 180, loss: 2348.85498046875
Epoch 0, iter 190, loss: 2337.46435546875
Epoch 0, iter 200, loss: 2047.4993896484375
Epoch 0, iter 210, loss: 1708.462890625
Epoch 0, iter 220, loss: 2057.809326171875
Epoch 0, iter 230, loss: 1366.5936279296875
Epoch 0, iter 240

In [187]:
torch.save(model.state_dict(), "keypoints_model.pth")